# (MBTI 별 Data 모으기: 생략) MBTI file 불러오기

In [2]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from collections import Counter

In [3]:
mbti=pd.read_csv('.\data\\training\\mbti.csv')

# UnderSampling and OverSampling

In [6]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

X_data = mbti['posts']
X = X_data.values.reshape(-1, 1)

y_data = mbti['type']
y = y_data.values.reshape(-1, 1)

X_resampled, y_resampled = RandomUnderSampler(random_state=0,
sampling_strategy={
         'ENFP': 500,
         'ENTP': 500,
         'INFJ': 500,
         'INFP': 500,
         'INTJ': 500,
         'INTP': 500,
         }).fit_sample(X, y)
X_resampled, y_resampled = RandomOverSampler(random_state=0).fit_sample(X_resampled, y_resampled)
#format(Counter(y_resampled))
X_resampled #ndarray
Counter(y_resampled)


Counter({'ENFJ': 500,
         'ENFP': 500,
         'ENTJ': 500,
         'ENTP': 500,
         'ESFJ': 500,
         'ESFP': 500,
         'ESTJ': 500,
         'ESTP': 500,
         'INFJ': 500,
         'INFP': 500,
         'INTJ': 500,
         'INTP': 500,
         'ISFJ': 500,
         'ISFP': 500,
         'ISTJ': 500,
         'ISTP': 500})

# tokenize and stemming

In [9]:
X_data = X_resampled.ravel()
y_data = y_resampled

print("normalize 시작")
normalized_text = []
for sentence in X_data: #for sentence in X_data.ravel()
    clean_sentence = re.sub('[^A-Za-z\s]+', '', sentence.lower())
    normalized_text.append(clean_sentence)
X_data = normalized_text
print("normalize 완료")

normalize 시작
normalize 완료


In [10]:
import nltk
import gc
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
ps=PorterStemmer()

# print("stopwords 제거 시작")
stopwords = ['http*']
#print("stopwords 제거 완료")

print("stemming 시작")
stemData=[]
for sentence in X_data:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []
    for word in tokenData:
        tempData.append(ps.stem(word))
        #tempData = [word for word in tempData if not word in stopwords] # 불용어 제거
    stemData.append(tempData)
print("stemming 완료")
#print(stemData[0])

#'list' object has no attribute 'lower' 문제 발생
#2차원 리스트인 stemData를 1차원 리스트로
#flat_stem = [item for sublist in stemData for item in sublist]
del [[mbti]]
gc.collect()

stemming 시작
stemming 완료


4

In [12]:
#vectorization (tfidf)
def dummy_fun(doc):
    return doc

tfidf=TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

tfidf.fit(stemData)
print(tfidf.transform(stemData).toarray())
X = tfidf.transform(stemData).toarray()
y = np.array(y_data)


[[0.11360815 0.         0.         ... 0.         0.         0.        ]
 [0.23612205 0.03721221 0.         ... 0.         0.         0.        ]
 [0.10170195 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.21440755 0.03379006 0.         ... 0.         0.         0.        ]
 [0.1786607  0.         0.         ... 0.         0.         0.        ]
 [0.23497077 0.         0.         ... 0.         0.         0.        ]]


# XGBoost_Undersampling

In [13]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

In [14]:
print("Model building Start")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=1234)
model = XGBClassifier()
model.fit(X_train, y_train)

Model building Start


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [15]:
preds = model.predict(X_test)
print("Model building End")

Model building End


In [16]:
import joblib

joblib.dump(model, 'XGBoost_undersample.pkl')

['XGBoost_undersample.pkl']

In [17]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print("Evaluation Start")
# labels과 guesses
labels = preds
guesses = y_test

print(accuracy_score(labels, guesses))
print(recall_score(labels, guesses, average=None))
print(precision_score(labels, guesses, average=None))
print(f1_score(labels, guesses, average=None))

print("Evaluation END")
# https://eunsukimme.github.io/ml/2019/10/21/Accuracy-Recall-Precision-F1-score/

Evaluation Start
0.5159574468085106
[0.66666667 0.55555556 0.54545455 0.17391304 0.88888889 0.46153846
 0.63636364 0.77777778 0.4        0.75       0.3        0.3
 0.61538462 0.5        0.55555556 0.5       ]
[0.75       0.38461538 0.75       0.4        0.5        0.54545455
 0.46666667 0.53846154 0.25       0.69230769 0.33333333 0.3
 0.53333333 0.7        0.55555556 0.54545455]
[0.70588235 0.45454545 0.63157895 0.24242424 0.64       0.5
 0.53846154 0.63636364 0.30769231 0.72       0.31578947 0.3
 0.57142857 0.58333333 0.55555556 0.52173913]
Evaluation END
